## NERD DOS DADOS - CRIANDO UM SISTEMA DE RECOMENDAÇÃO DE FILMES <br>
<p> Fonte de dados: https://www.kaggle.com/code/alyssonbispopereira/recomenda-o-de-filmes-ptbr/data

In [1]:
# Importando os pacotes a serem utilizados
import pandas as pd
import numpy as np

In [2]:
# Importar o arquivo com os filmes e visualizar as primeiras linhas
filmes = pd.read_csv('movies_metadata.csv', low_memory = False)
filmes.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [ ]:
# Importando o arquivo de avaliações e avaliando as primeiras linhas
avaliacoes = pd.read_csv('ratings.csv')
avaliacoes.head()

In [1]:
# Importando os pacotes a serem utilizados
import pandas as pd
import numpy as np
import ast

# Importar o arquivo com os filmes e visualizar as primeiras linhas
filmes = pd.read_csv('movies_metadata.csv', low_memory = False)
def generos(df):
    if isinstance(df, str):
        df=ast.literal_eval(df)
        nomes=[item['name']for item in df]
        return nomes
    return []
def pegar_nome(df):
    try:
        companies=ast.literal_eval(df)
        nomes=[company['nomes']for company in companies]
        return nomes
    except(ValueError, TypeError):
        return[]
filmes['genres'] = filmes['genres'].apply(generos)

# Importando o arquivo de avaliações e avaliando as primeiras linhas
avaliacoes = pd.read_csv('ratings.csv')

# Seleciona somente as variaveis que iremos utilizar
filmes = filmes [['id','original_title','original_language','vote_count']]

# Renomeia as variaveis
filmes.rename(columns = {'id':'ID_FILME','original_title':'TITULO','original_language':'LINGUAGEM','vote_count':'QT_AVALIACOES'}, inplace = True)

# Seleciona somente as variaveis que iremos utilizar
avaliacoes = avaliacoes [['userId','movieId','rating']]

# Renomeia as variaveis
avaliacoes.rename(columns = {'userId':'ID_USUARIO','movieId':'ID_FILME','rating':'AVALIACAO'}, inplace = True)

# Verificando se há valores nulos
filmes.isna().sum()

# Como são poucos os valores nulos iremos remover porque não terá impacto nenhum
filmes.dropna(inplace = True)

# Verificando se há valores nulos
filmes.isna().sum()

# Verificando se há valores nulos
avaliacoes.isna().sum()

# Verificando a quantidade de avaliacoes por usuarios
avaliacoes['ID_USUARIO'].value_counts()

# Vamos pegar o ID_USUARIO somente de usuários que fizeram mais de 999 avaliações
qt_avaliacoes = avaliacoes['ID_USUARIO'].value_counts() > 999
y = qt_avaliacoes[qt_avaliacoes].index

# Pegando somente avaliacoes dos usuarios que avaliaram mais de 999 vezes
avaliacoes = avaliacoes[avaliacoes['ID_USUARIO'].isin(y)]



# Vamos usar os filmes que possuem somente uma quantidade de avaliações superior a 999 avaliações
filmes = filmes[filmes['QT_AVALIACOES'] > 999]

# Vamos agrupar e visualizar a quantidade de filmes pela linguagem
filmes_linguagem = filmes['LINGUAGEM'].value_counts()


# Selecionar somente os filmes da linguagem EN (English)
filmes = filmes[filmes['LINGUAGEM'] == 'en']

# Visualizar os tipos de dados das variaveis
filmes.info()

avaliacoes.info()

# Precisamos converter a variavel ID_FILME em inteiro
filmes['ID_FILME'] = filmes['ID_FILME'].astype(int)

filmes.info()

# Concatenando os dataframes
avaliacoes_e_filmes = avaliacoes.merge(filmes, on = 'ID_FILME')

# Verificando se há valores nulos
avaliacoes_e_filmes.isna().sum()



# Vamos descartar os valores duplicados, para que não tenha problemas de termos o mesmo usuário avaliando o mesmo filme
# diversas vezes
avaliacoes_e_filmes.drop_duplicates(['ID_USUARIO','ID_FILME'], inplace = True)
display(avaliacoes_e_filmes)

# Vamos excluir a variavel ID_FILME porque não iremos utiliza-la
del avaliacoes_e_filmes['ID_FILME']

# DataFrame sem a variavel ID_FILME
display(avaliacoes_e_filmes.loc[avaliacoes_e_filmes['AVALIACAO'] ==5.0])

# Agora precisamos fazer um PIVOT. O que queremos é que cada ID_USUARIO seja uma variavel com o respectivo valor de nota
# para cada filme avaliado
filmes_pivot = avaliacoes_e_filmes.pivot_table(columns = 'ID_USUARIO', index = 'TITULO', values = 'AVALIACAO')




# Os valores que são nulos iremos preencher com ZERO
filmes_pivot.fillna(0, inplace = True)


# Vamos importar o csr_matrix do pacote SciPy
# Esse método possibilita criarmos uma matriz sparsa
from scipy.sparse import csr_matrix


# Vamos transformar o nosso dataset em uma matriz sparsa
filmes_sparse = csr_matrix(filmes_pivot)

# Tipo do objeto
type(filmes_sparse)

# Vamos importar o algoritmo KNN do SciKit Learn
from sklearn.neighbors import NearestNeighbors 

# Criando e treinando o modelo preditivo
modelo = NearestNeighbors(algorithm = 'brute')
modelo.fit(filmes_sparse)



## Vamos fazer previsões de sugestões de filmes

# 127 Hours
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['127 Hours'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]])  

# Toy Story
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['Toy Story'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]])  

# 1408
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['1408'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]]) 

# 2 Fast 2 Furious
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['2 Fast 2 Furious'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]]) 




<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 44842
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_FILME       1100 non-null   object 
 1   TITULO         1100 non-null   object 
 2   LINGUAGEM      1100 non-null   object 
 3   QT_AVALIACOES  1100 non-null   float64
dtypes: float64(1), object(3)
memory usage: 43.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3844582 entries, 17291 to 26023521
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ID_USUARIO  int64  
 1   ID_FILME    int64  
 2   AVALIACAO   float64
dtypes: float64(1), int64(2)
memory usage: 117.3 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 44842
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_FILME       1100 non-null   int32  
 1   TITULO         1100 non-null   object 

,ID_USUARIO,ID_FILME,AVALIACAO,TITULO,LINGUAGEM,QT_AVALIACOES
0,229,12,1.0,Finding Nemo,en,6292.0
1,741,12,3.0,Finding Nemo,en,6292.0
2,1932,12,0.5,Finding Nemo,en,6292.0
3,3437,12,2.0,Finding Nemo,en,6292.0
4,3694,12,1.5,Finding Nemo,en,6292.0
...,...,...,...,...,...,...
189877,243443,8909,1.0,Wanted,en,2583.0
189878,218368,101299,4.0,The Hunger Games: Catching Fire,en,6656.0
189879,219677,146216,1.0,RED 2,en,1548.0
189880,221327,118340,4.0,Guardians of the Galaxy,en,10014.0


,ID_USUARIO,AVALIACAO,TITULO,LINGUAGEM,QT_AVALIACOES
11,5620,5.0,Finding Nemo,en,6292.0
419,195282,5.0,Finding Nemo,en,6292.0
441,210231,5.0,Finding Nemo,en,6292.0
579,4384,5.0,Million Dollar Baby,en,2519.0
607,8233,5.0,Million Dollar Baby,en,2519.0
...,...,...,...,...,...
189675,74370,5.0,Hitch,en,1721.0
189735,125288,5.0,Apocalypto,en,1252.0
189784,144616,5.0,The Usual Suspects,en,3334.0
189785,204641,5.0,The Usual Suspects,en,3334.0


Index(['127 Hours', 'American Hustle', 'The Expendables 2', 'Lord of War',
       'RED 2'],
      dtype='object', name='TITULO')
Index(['Toy Story', 'Meet the Fockers', 'Top Gun',
       'Harry Potter and the Chamber of Secrets',
       'Austin Powers: International Man of Mystery'],
      dtype='object', name='TITULO')
Index(['1408', 'Pirates of the Caribbean: At World's End', 'Platoon', 'Snitch',
       'The Expendables 2'],
      dtype='object', name='TITULO')
Index(['2 Fast 2 Furious', 'Bambi', 'The Matrix Reloaded',
       'Brokeback Mountain', 'Lord of War'],
      dtype='object', name='TITULO')


## Pré Processamento dos Dados

In [ ]:
# Filtrando somente as colunas necessários e renomeando nome das variaveis

# Seleciona somente as variaveis que iremos utilizar
filmes = filmes [['id','original_title','original_language','vote_count']]

# Renomeia as variaveis
filmes.rename(columns = {'id':'ID_FILME','original_title':'TITULO','original_language':'LINGUAGEM','vote_count':'QT_AVALIACOES'}, inplace = True)

# Exibe as primeiras linhas do arquivo tratado
filmes.head()

In [ ]:
# Filtrando somente as colunas necessários e renomeando nome das variaveis

# Seleciona somente as variaveis que iremos utilizar
avaliacoes = avaliacoes [['userId','movieId','rating']]

# Renomeia as variaveis
avaliacoes.rename(columns = {'userId':'ID_USUARIO','movieId':'ID_FILME','rating':'AVALIACAO'}, inplace = True)

# Exibe as primeiras linhas do arquivo tratado
avaliacoes.head()

In [ ]:
# Verificando se há valores nulos
filmes.isna().sum()

In [ ]:
# Como são poucos os valores nulos iremos remover porque não terá impacto nenhum
filmes.dropna(inplace = True)

In [ ]:
# Verificando se há valores nulos
filmes.isna().sum()

In [ ]:
# Verificando se há valores nulos
avaliacoes.isna().sum()

In [ ]:
# Verificando a quantidade de avaliacoes por usuarios
avaliacoes['ID_USUARIO'].value_counts()

In [ ]:
# Vamos pegar o ID_USUARIO somente de usuários que fizeram mais de 999 avaliações
qt_avaliacoes = avaliacoes['ID_USUARIO'].value_counts() > 999
y = qt_avaliacoes[qt_avaliacoes].index
y.shape

In [ ]:
# Visualizando os usuarios selecionados
y

In [ ]:
# visualizando o tamanho do dataset Avaliações
avaliacoes.shape

In [ ]:
# Pegando somente avaliacoes dos usuarios que avaliaram mais de 999 vezes
avaliacoes = avaliacoes[avaliacoes['ID_USUARIO'].isin(y)]

In [ ]:
# visualizando o tamanho do dataset Avaliações
avaliacoes.shape

In [ ]:
# Visualizando os DataFrame Avaliacoes
avaliacoes.head()

In [ ]:
# Visualizando o DataFrame Filmes
filmes.head()

In [ ]:
# Vamos usar os filmes que possuem somente uma quantidade de avaliações superior a 999 avaliações
filmes = filmes[filmes['QT_AVALIACOES'] > 999]

In [ ]:
# Vamos agrupar e visualizar a quantidade de filmes pela linguagem
filmes_linguagem = filmes['LINGUAGEM'].value_counts()
filmes_linguagem.head(20)

In [ ]:
# Selecionar somente os filmes da linguagem EN (English)
filmes = filmes[filmes['LINGUAGEM'] == 'en']

In [ ]:
# Visualizar os tipos de dados das variaveis
filmes.info()

In [ ]:
avaliacoes.info()

In [ ]:
# Precisamos converter a variavel ID_FILME em inteiro
filmes['ID_FILME'] = filmes['ID_FILME'].astype(int)

In [ ]:
# Verificando a quantidade de filmes pelo tamanho do arquivo
filmes.shape

In [ ]:
filmes.info()

In [ ]:
# Concatenando os dataframes
avaliacoes_e_filmes = avaliacoes.merge(filmes, on = 'ID_FILME')
avaliacoes_e_filmes.head()

In [ ]:
# Verificando a quantidade de filmes com avaliacoes pelo tamanho do arquivo
avaliacoes_e_filmes.shape

In [ ]:
# Verificando se há valores nulos
avaliacoes_e_filmes.isna().sum()

In [ ]:
# Vamos visualizar as primeiras 20 linhas do arquivo
avaliacoes_e_filmes.head(20)

In [ ]:
# Vamos descartar os valores duplicados, para que não tenha problemas de termos o mesmo usuário avaliando o mesmo filme
# diversas vezes
avaliacoes_e_filmes.drop_duplicates(['ID_USUARIO','ID_FILME'], inplace = True)
display(avaliacoes_e_filmes)

In [ ]:
# Visualizando se houve alteração na quantidade de registros
avaliacoes_e_filmes.shape

In [ ]:
# Vamos excluir a variavel ID_FILME porque não iremos utiliza-la
del avaliacoes_e_filmes['ID_FILME']

In [ ]:
# DataFrame sem a variavel ID_FILME
avaliacoes_e_filmes

In [ ]:
# Agora precisamos fazer um PIVOT. O que queremos é que cada ID_USUARIO seja uma variavel com o respectivo valor de nota
# para cada filme avaliado
filmes_pivot = avaliacoes_e_filmes.pivot_table(columns = 'ID_USUARIO', index = 'TITULO', values = 'AVALIACAO')


# Avaliar o arquivo transformado para PIVOT 
filmes_pivot.head(20)

In [ ]:
# Os valores que são nulos iremos preencher com ZERO
filmes_pivot.fillna(0, inplace = True)
filmes_pivot.head()

In [ ]:
# Vamos importar o csr_matrix do pacote SciPy
# Esse método possibilita criarmos uma matriz sparsa
from scipy.sparse import csr_matrix


# Vamos transformar o nosso dataset em uma matriz sparsa
filmes_sparse = csr_matrix(filmes_pivot)

In [ ]:
# Tipo do objeto
type(filmes_sparse)

In [ ]:
# Vamos importar o algoritmo KNN do SciKit Learn
from sklearn.neighbors import NearestNeighbors 

In [ ]:
# Criando e treinando o modelo preditivo
modelo = NearestNeighbors(algorithm = 'brute')
modelo.fit(filmes_sparse)

## Vamos fazer previsões de sugestões de filmes

In [ ]:
# 127 Hours
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['127 Hours'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]])  

In [ ]:
# Toy Story
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['Toy Story'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]])  

In [ ]:
# 1408
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['1408'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]]) 

In [ ]:
# 2 Fast 2 Furious
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['2 Fast 2 Furious'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]]) 